In [1]:
import numpy as np
import itertools
import psycopg2

# Base de Datos y Variables de Entorno
from conexion import connect_to_postgresql

In [2]:
engine = connect_to_postgresql()
conn = engine.raw_connection()
cur = conn.cursor()
try:
    
    ## cudis

    query = """
    SELECT DISTINCT mun."CUDIS" FROM dim_municipios mun;
    """
    cur.execute(query)
    cudis = np.array(cur.fetchall())
    conn.commit()
    print("CUDIS cargada correctamente como array de NumPy")

    ## municipio_tipo_de_vivienda
    query = """
    SELECT DISTINCT ind."Municipio_Tipo" FROM ft_indicadores ind;
    """
    cur.execute(query)
    municipio_tipo = np.array(cur.fetchall())
    conn.commit()
    print("municipio_tipo cargada correctamente como array de NumPy")

    ## habitaciones
    query = """
    SELECT DISTINCT ind."Habitaciones" FROM ft_indicadores ind;
    """
    cur.execute(query)
    Habitaciones = np.array(cur.fetchall())
    conn.commit()
    print("Habitaciones cargada correctamente como array de NumPy")

    ## aseos
    query = """
    SELECT DISTINCT ind."Aseos" FROM ft_indicadores ind;
    """
    cur.execute(query)
    Aseos = np.array(cur.fetchall())
    conn.commit()
    print("Aseos cargada correctamente como array de NumPy")

    ## Metros
    query = """
    SELECT generate_series(40, (SELECT MAX(ind."Metros")::integer FROM ft_indicadores ind), 10) AS "Metros";
    """
    cur.execute(query)
    Metros = np.array(cur.fetchall())
    conn.commit()
    print("Metros cargada correctamente como array de NumPy")

    ## Comodidades
    query = """
    SELECT com."Comodidades" FROM comodidades com;
    """
    cur.execute(query)
    comodidades = np.array(cur.fetchall())
    conn.commit()
    print("comodidades cargada correctamente como array de NumPy")

except Exception as e:
    print(f"Ha ocurrido un error: {str(e)}")
finally:
    cur.close()
    conn.close()

Conexión establecida
CUDIS cargada correctamente como array de NumPy
municipio_tipo cargada correctamente como array de NumPy
Habitaciones cargada correctamente como array de NumPy
Aseos cargada correctamente como array de NumPy
Metros cargada correctamente como array de NumPy
comodidades cargada correctamente como array de NumPy


In [3]:
def transformar_tipo(tipo):
    if tipo.endswith("Apartment"):
        return "Piso"
    elif tipo.endswith("Single-family Home"):
        return "Casa"
    else:
        return tipo  # Mantiene otros valores sin cambios

# Aplicamos la transformación al array
transformar_vectorizado = np.vectorize(transformar_tipo)
municipio_tipo = transformar_vectorizado(municipio_tipo)
municipio_tipo = np.unique(municipio_tipo)

In [4]:
# Función para convertir valores numéricos a enteros cuando sea necesario
def convert_value(value):
    """Convierte valores numéricos correctamente evitando problemas con arrays."""
    try:
        if isinstance(value, np.ndarray):  
            value = value.item()  # Extrae el elemento único si es un array

        return int(float(value))  # Convierte correctamente evitando errores
    except ValueError:
        return value  # Si no es un número, lo mantiene como string


# Función para generar combinaciones sin NMUN
def generate_combinations():
    for combo in itertools.product(
        map(convert_value, cudis),  # CUDIS
        municipio_tipo.flatten(),  # Municipio_Tipo
        map(convert_value, Habitaciones.flatten()),  # Habitaciones
        map(convert_value, Aseos.flatten()),  # Aseos
        map(convert_value, Metros.flatten()),  # Metros
        comodidades.flatten()  # Solo la columna Comodidades
    ):
        yield combo  # Generamos fila por fila sin almacenar todo en memoria

# Conectamos a la base de datos
try:
    engine = connect_to_postgresql()
    conn = engine.raw_connection()
    cur = conn.cursor()

    batch_size = 100000
    batch = []
    
    for row in generate_combinations():
        batch.append(row)
        
        # Insertamos en batches de 10,000 registros
        if len(batch) >= batch_size:
            query = """
            INSERT INTO dim_propiedades_api (CUDIS, Municipio_Tipo, Habitaciones, Aseos, Metros, Comodidades) 
            VALUES %s
            """
            psycopg2.extras.execute_values(cur, query, batch)
            conn.commit()
            batch.clear()  # Limpiamos el lote después de la inserción
    
    # Insertamos cualquier dato restante que no haya completado un lote
    if batch:
        psycopg2.extras.execute_values(cur, query, batch)
        conn.commit()
    
    print("Datos insertados en la base de datos correctamente.")

except Exception as e:
    print(f"Error al insertar datos: {e}")

finally:
    cur.close()
    conn.close()

Conexión establecida
Datos insertados en la base de datos correctamente.
